# icecream

The Annapolis Ice Cream Company is small and often overcrowded.
If there are too many customers already in line at the shop, then arriving customers will go elsewhere.
In addition, customers are rather impatient, and will abandon the line without getting served if they've waited too long.

The interarrival time between potential customers is exponentially distributed with a mean of 3 minutes.
There is 1 server, and the service time is exponentially distributed with a mean of 6 minutes.
The probability that a customer arrive at the shop and then go elsewhere is $\frac{4e^x}{5e^x + 100}$ when there are $x$ customers already in line.
The time that a customer will spend in line before abandoning it is exponentially distributed with mean 10 minutes.

* Write a SimPy model that simulates the shop for 8 hours.


* Using 100 replications, compute point and interval estimates for the mean fraction of customers that arrive at the shop and leave without joining the line.


* Using 100 replications, compute point and interval estimates for the mean fraction of customers that join the line at the shop, but then abandon the line without getting served.


*Hint.* You will find the [`exp`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html) function in NumPy useful.

First, some setup code.

In [1]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import mean, std, and sqrt from NumPy
from numpy import mean, std, sqrt, exp

# Import RandomState from numpy.random
from numpy.random import RandomState, rand

# Import t random variable object from scipy.stats
from scipy.stats import t

The SimPy model:

In [2]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 3
    interarrivalTimeMean = 3
    
    # Service times are exponentially distributed with mean 6 minutes
    serviceTimeMean = 6
    
    # Reneging times are exponentially distributed with mean 10 minutes
    renegeTimeMean = 10
    
    # One server
    nServers = 1
    
    # Shop is open for 8 continuous hours
    simulationTimeMax = 8 * 60
    
    
##### Streams #####
class S:
    inter = RandomState()
    serve = RandomState()
    renege = RandomState()
 
    
##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Get current number of customers in queue
        x = len(R.server.waitQ)
        
        # Customer doesn't balk...
        if rand() > (4 * exp(x)) / (5 * exp(x) + 100):
            M.balk.observe(0)
            
            # Customer arrives, joins queue, waits until service 
            # or renege time limit, whichever comes first
            renegeTime = S.renege.exponential(scale = P.renegeTimeMean)
            yield (request, self, R.server), (hold, self, renegeTime)
            
            # Customer doesn't renege...
            if self.acquired(R.server):
                M.renege.observe(0)
        
                # Customer is released from queue and starts service
                serviceTime = S.serve.exponential(scale = P.serviceTimeMean)
                yield hold, self, serviceTime
        
                # Customer finishes service, leaves
                yield release, self, R.server
                
            # Customer reneges...
            else:
                M.renege.observe(1)
        
        # Customer balks....
        else:
            M.balk.observe(1)

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = S.inter.exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Server
    server = None

    
##### Monitors #####
class M:
    balk = None
    renege = None

    
##### Model #####
def model(interSeed, serveSeed, renegeSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the interarrival time random number stream
    S.inter.seed(interSeed)
    
    # Initialize a seed for the service time random number stream
    S.serve.seed(serveSeed)
    
    # Initialize a seed for the reneging time random number stream
    S.renege.seed(renegeSeed)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)
    
    # Create monitor for balking customers
    M.balk = Monitor()
    
    # Create monitor for reneging customers
    M.renege = Monitor()
    
    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)
    
    # Compute fraction of customers that balk
    fracBalk = M.balk.mean()
    
    # Compute fraction of customers that renege
    fracRenege = M.renege.mean()
    
    # Return both performance measures
    return [fracBalk, fracRenege]

With the model in hand, we can replicate the simulation and compute point and interval estimates.

In [3]:
# Replicate simulation 100 times
# Create a list of observations for each performance measure
n = 100
fracBalkObs = []
fracRenegeObs = []
for i in range(n):
    [fracBalk, fracRenege] = model(123*i, 456*i, 789*i)
    fracBalkObs.append(fracBalk)
    fracRenegeObs.append(fracRenege)

    
#### Fraction of customers that balk ####
# Observed sample mean
fracBalkSM = mean(fracBalkObs)

# Observed sample standard deviation
fracBalkSSD = std(fracBalkObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
fracBalkCIL = fracBalkSM - t.ppf(1 - alpha/2, n - 1) * fracBalkSSD / sqrt(n)
fracBalkCIR = fracBalkSM + t.ppf(1 - alpha/2, n - 1) * fracBalkSSD / sqrt(n)

print("Fraction of customers that balk:")
print("  Sample mean: {0}".format(fracBalkSM))
print("  Sample standard deviation: {0}".format(fracBalkSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, fracBalkCIL, fracBalkCIR))


#### Fraction of customers that renege ####
# Observed sample mean
fracRenegeSM = mean(fracRenegeObs)

# Observed sample standard deviation
fracRenegeSSD = std(fracRenegeObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
fracRenegeCIL = fracRenegeSM - t.ppf(1 - alpha/2, n - 1) * fracRenegeSSD / sqrt(n)
fracRenegeCIR = fracRenegeSM + t.ppf(1 - alpha/2, n - 1) * fracRenegeSSD / sqrt(n)

print("Fraction of customers that renege:")
print("  Sample mean: {0}".format(fracRenegeSM))
print("  Sample standard deviation: {0}".format(fracRenegeSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, fracRenegeCIL, fracRenegeCIR))

Fraction of customers that balk:
  Sample mean: 0.16614790999249113
  Sample standard deviation: 0.03456164301630732
  95.0% confidence interval: [0.15929013019799626, 0.173005689786986]
Fraction of customers that renege:
  Sample mean: 0.4573549103070161
  Sample standard deviation: 0.049125040484470965
  95.0% confidence interval: [0.44760743649973234, 0.46710238411429983]
